In [7]:

import numpy as np
import pandas as pd 
import tensorflow.keras.backend as K
from datetime import datetime, timedelta
import keras
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Reshape, concatenate, Input, Lambda, Flatten, Activation

In [8]:
"""Variables (+ Hyperparams)"""
predict_length = 10
look_ahead = 20

In [13]:
"""seq2seq model attempt 1"""
input = tf.keras.Input(shape=(1, predict_length))

class SingleDayModelInstance(keras.layers.Layer):
    def __init__(self):
        super().__init__()
        
    def call(self, input):
        lstm1 = LSTM(units=60, return_sequences = True, input_shape=(input.shape[1],1))(input)
        #drop1 = Dropout(0.2)(lstm1)
        lstm2 = LSTM(units=35, return_sequences = False)(lstm1)
        dense1 = Dense(units=30)(lstm2)
        drop2 = Dropout(0.2)(dense1)
        dense2 = Dense(units = 1)(drop2)
        return dense2

# Represents a copy of model:
single_day_model = SingleDayModelInstance()#dense2

#inputs = []
#inputs = np.array(input)
inputs = input
#outputs = np.array([])#[single_day_model(input)]
outputs_raw = K.variable([])
#inputs = inputs.eval()
for i in range(look_ahead):
    concatted = concatenate([inputs, outputs_raw])
    #cur_input = concatenate([inputs[i:], outputs])
    cur_input = concatted[i:i+predict_length]
    outputs_raw.append(single_day_model(cur_input))
    #cur_layer = concatenate(outputs[i-1], )
    #inputs= inputs.append(out)
    #inputs = np.delete(inputs,0)
    #inputs = K.constant(inputs)

model = tf.keras.Model(inputs = input, outputs = outputs_raw, name = "test")
'''
X_train, Y_train = self.train_2(self.predict_length, self.look_ahead)
model.fit(X_train, Y_train, epochs=1, batch_size=32)
'''


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 1, 10), (0,)]

In [58]:
"""seq2seq model attempt 1"""
input = tf.keras.Input(shape=(predict_length, 1))

class SingleDayModelInstance(keras.layers.Layer):
    def __init__(self):
        #input = Input(shape=(1, predict_length))
        self.lstm1 = LSTM(units=60, return_sequences = True, input_shape=(input.shape))
        #drop1 = Dropout(0.2)(lstm1)
        self.lstm2 = LSTM(units=35, return_sequences = False)
        self.dense1 = Dense(units=30)
        self.drop2 = Dropout(0.2)
        self.dense2 = Dense(units = 1)
        self.reshape1 = Reshape((1, 1))
        super().__init__()
        
    def call(self, input):
        x = self.lstm1(input)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.drop2(x)
        x = self.dense2(x)
        x = self.reshape1(x)

        return x#np.reshape(x, (x.shape, 1))

# Represents a copy of model:
single_day_model = SingleDayModelInstance()#dense2
outputs_raw = single_day_model(input)
#outputs = [single_day_model(input)]

for i in range(1, look_ahead):
    concatted = concatenate([input, outputs_raw], axis=1)
    cur_input = concatted[i:i+predict_length]
    outputs_raw = concatenate([outputs_raw, single_day_model(cur_input)], axis=1)
    #outputs.append(single_day_model(cur_input))
print(outputs_raw.shape)
#concatted = concatenate([input, outputs[0]], axis=1) # np.squeeze(input) WORKS

# Reshape outputs
outputs = Reshape((-1,))(outputs_raw)
print(outputs.shape)
'''
#inputs = []
#inputs = np.array(input)
inputs = input
#outputs = np.array([])#[single_day_model(input)]
outputs = K.variable([])
#inputs = inputs.eval()
for i in range(look_ahead):
    concatted = concatenate([inputs, outputs])
    #cur_input = concatenate([inputs[i:], outputs])
    cur_input = concatted[i:i+predict_length]
    outputs.append(single_day_model(cur_input))
    #cur_layer = concatenate(outputs[i-1], )
    #inputs= inputs.append(out)
    #inputs = np.delete(inputs,0)
    #inputs = K.constant(inputs)
'''
model = tf.keras.Model(inputs = input, outputs = outputs, name = "test")
'''
X_train, Y_train = self.train_2(self.predict_length, self.look_ahead)
model.fit(X_train, Y_train, epochs=1, batch_size=32)
'''
print(model.summary())


(None, 20, 1)
(None, 20)
Model: "test"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_56 (InputLayer)       [(None, 10, 1)]              0         []                            
                                                                                                  
 single_day_model_instance_  (None, 1, 1)                 29431     ['input_56[0][0]',            
 45 (SingleDayModelInstance                                          'tf.__operators__.getitem_233
 )                                                                  [0][0]',                      
                                                                     'tf.__operators__.getitem_234
                                                                    [0][0]',                      
                                                                     '